# Data Import & Connection to BigQuery

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from google.cloud import bigquery
from itertools import product
from functions import *

# set display options to show all columns
pd.set_option('display.max_columns', None)
# Set the float format to display numbers without scientific notation
pd.options.display.float_format = '{:.2f}'.format
# Set the client for future queries to BigQuery
client = bigquery.Client(project = "continente-lced-feup")

C:\Users\luish\anaconda3\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=Dr2IITnrICIQrJOOzN9Z7Hmj4bmhJR&access_type=offline&code_challenge=WAX8C2fRYBfGpW4cfLnvzIqFWjlOuAmgl704PtbszGc&code_challenge_method=S256


Credentials saved to file: [C:\Users\luish\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot find a quota project to add to ADC. You might receive a "quota exceeded" or "API not enabled" error. Run $ gcloud auth application-default set-quota-project to add a 

# Data Loading

In [43]:
query = client.query("""
   SELECT *
   FROM 
       tables_raw.dim_customer 
       JOIN tables_raw.fact_transaction USING(CUSTOMER_ACCOUNT_NR_MASK)
       JOIN tables_raw.dim_product USING(SKU)
    WHERE 
        SUBCAT_CD_EXT IN (140304, 50401, 30301, 20201, 10301, 80103, 60102, 60401, 30401, 10101, 100102, 100204, 140204, 
        20302, 30201, 50203, 90201, 170101, 80404, 50402, 70202, 100201, 80409, 10303, 20306, 80411, 10102, 20305, 60105, 80110, 
        140301, 30202, 90202, 100101, 80105, 80104, 50202, 50303, 70204, 60306, 80403, 10302, 10201, 80405, 170304, 170303, 80406, 
        140201, 60302, 30403, 30304, 20204, 170106, 140205, 10204, 60404, 50301, 50302, 20205, 60406, 20301, 80407, 20203, 70201, 100205,
        60106, 170302, 50201, 60301, 10205, 30203, 80401, 100202, 30302, 170111, 10202, 70203, 60303, 170109, 60403, 30402, 140302, 30208, 60307, 
        80107, 50403, 60103, 20307, 60305, 60101, 170307, 80414, 80415, 60405, 20303, 80402, 30204, 30206, 170310, 60304, 140206, 10203, 30205, 60107, 
        70206, 170108, 90203, 90204, 30207, 140303, 30303, 80408, 140202, 50304, 80101, 170313, 100203, 60402, 170305, 50305, 50404, 20202, 170110, 
        170105, 170112, 170301, 10206, 10208, 20304, 80102, 70205, 10207, 10305, 170309, 170114, 80111, 90206, 30306, 30305, 140203, 80413) 
        AND SEG_AGE_DSC = ']25;35]'
        AND QTY >= 0
    ORDER BY CUSTOMER_ACCOUNT_NR_MASK DESC, TIME_KEY ASC
   """)

df = query.result().to_dataframe() # Wait for the job to complete.

# Data Preparation (more to be done...)

In [44]:
df = df.drop(columns=['LOC_BRAND_CD','PROD_DSCNT_ISSUED_AMT','NET_SLS_AMT','TRANS_DSCNT_RAT_AMT','DIRECT_DSCNT_AMT',
                 'seg_lifestyle_dsc','SEG_AGE','SEG_AGE_DSC','seg_lifestage_dsc',
                 'UNIT_BASE_DSC_EXT','SUBCAT_DSC_EXT','BIZ_UNIT_DSC_EXT','DEPARTMENT_DSC_EXT',
                'PRODUCT_SHORT_DSC','BRAND_DSC','BRAND_TYPE_CD','CONVERSION_FACTOR','CAPACITY_UNIT','PRODUCT_DSC','SKU',
                'LOCATION_CD','GROSS_SLS_AMT','CP4','CAT_DSC_EXT','PRODUCT_KEY','POS_TP_CD','PRICE_RANGE'])

In [45]:
df.isnull().sum()

CUSTOMER_ACCOUNT_NR_MASK         0
GENDER                      207890
FAMILY_MEMBERS              778011
seg_lifestyle_cd                 0
seg_lifestage_cd                 0
TIME_KEY                         0
TRANSACTION_ID_MASK              0
POS_TP_CD                        0
QTY                              0
UNIT_BASE_CD_EXT                 0
SUBCAT_CD_EXT                    0
CAT_CD_EXT                       0
BIZ_UNIT_CD_EXT                  0
DEPARTMENT_CD_EXT                0
PRICE_RANGE                      0
dtype: int64

# Feature Engineering & ML Dataset Development

In [46]:
# convert the 'TIME_KEY' column to datetime format
df['TIME_KEY'] = pd.to_datetime(df['TIME_KEY'], format='%Y%m%d')

# create new columns for the day, week, day of the week, month, quarter, and year
#df['DAY'] = df['TIME_KEY'].dt.day
#df['WEEK'] = df['TIME_KEY'].dt.week
#df['DOW'] = df['TIME_KEY'].dt.dayofweek
df['MONTH'] = df['TIME_KEY'].dt.month
df['QUARTER'] = df['TIME_KEY'].dt.quarter
df['SEMESTER'] = df['MONTH'].apply(semester)
df['YEAR'] = df['TIME_KEY'].dt.year

In [47]:
# Filter the dataframe by the customer's frequency score
df = filter_customers(df)
# Sort the dataframe (later suffled again)
df = df.sort_values(['TIME_KEY','TRANSACTION_ID_MASK','YEAR','MONTH'])

In [48]:
# get all unique values of customer_id, category_id, month and year
customer_ids = df['CUSTOMER_ACCOUNT_NR_MASK'].unique()
category_ids = df['SUBCAT_CD_EXT'].unique()
months = df['MONTH'].unique()
years = df['YEAR'].unique()

# create a new dataframe with all possible combinations of customer_id and category_id
ml_dataset = pd.DataFrame(list(product(customer_ids, category_ids, months, years)), 
                                    columns=['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT','MONTH','YEAR'])

# add the quarter and semester columns based on the month value
quarter_map = {1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 2, 7: 3, 8: 3, 9: 3, 10: 4, 11: 4, 12: 4}
semester_map = {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2, 12: 2}

ml_dataset['QUARTER'] = ml_dataset['MONTH'].map(quarter_map)
ml_dataset['SEMESTER'] = ml_dataset['MONTH'].map(semester_map)

In [9]:
# create a random sample of 1M rows to test the DM pipeline
#ml_dataset = ml_dataset.sample(n=1000000)
ml_dataset = ml_dataset.sort_values(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT','YEAR','MONTH'])

In [10]:
# TOTAL NUMBER OF ORDERS

df['CUST_NUM_TRANSACTIONS_MONTH'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR', 'MONTH'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['CUST_NUM_TRANSACTIONS_MONTH'] = df['CUST_NUM_TRANSACTIONS_MONTH'].astype(int)

df['CUST_NUM_TRANSACTIONS_QUARTER'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR', 'QUARTER'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['CUST_NUM_TRANSACTIONS_QUARTER'] = df['CUST_NUM_TRANSACTIONS_QUARTER'].astype(int)

df['CUST_NUM_TRANSACTIONS_SEMESTER'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR', 'SEMESTER'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['CUST_NUM_TRANSACTIONS_SEMESTER'] = df['CUST_NUM_TRANSACTIONS_SEMESTER'].astype(int)

df['CUST_NUM_TRANSACTIONS_YEAR'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['CUST_NUM_TRANSACTIONS_YEAR'] = df['CUST_NUM_TRANSACTIONS_YEAR'].astype(int)

ml_dataset = create_aggregations(df, ml_dataset, 'MONTH', 'CUST_NUM_TRANSACTIONS', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, 'QUARTER', 'CUST_NUM_TRANSACTIONS', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, 'SEMESTER', 'CUST_NUM_TRANSACTIONS', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, 'YEAR', 'CUST_NUM_TRANSACTIONS', 'CUST', False)

In [11]:
# UNIQUE NUMBER OF SUBCATEGORIES BOUGHT

df['CUST_NUM_UNIQUE_SUBCAT_MONTH'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR', 'MONTH'])['SUBCAT_CD_EXT'].transform(count_unique)
df['CUST_NUM_UNIQUE_SUBCAT_MONTH'] = df['CUST_NUM_UNIQUE_SUBCAT_MONTH'].astype(int)

df['CUST_NUM_UNIQUE_SUBCAT_QUARTER'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR', 'QUARTER'])['SUBCAT_CD_EXT'].transform(count_unique)
df['CUST_NUM_UNIQUE_SUBCAT_QUARTER'] = df['CUST_NUM_UNIQUE_SUBCAT_QUARTER'].astype(int)

df['CUST_NUM_UNIQUE_SUBCAT_SEMESTER'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR', 'SEMESTER'])['SUBCAT_CD_EXT'].transform(count_unique)
df['CUST_NUM_UNIQUE_SUBCAT_SEMESTER'] = df['CUST_NUM_UNIQUE_SUBCAT_SEMESTER'].astype(int)

df['CUST_NUM_UNIQUE_SUBCAT_YEAR'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR'])['SUBCAT_CD_EXT'].transform(count_unique)
df['CUST_NUM_UNIQUE_SUBCAT_YEAR'] = df['CUST_NUM_UNIQUE_SUBCAT_YEAR'].astype(int)


ml_dataset = create_aggregations(df, ml_dataset, 'MONTH', 'CUST_NUM_UNIQUE_SUBCAT', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, 'QUARTER', 'CUST_NUM_UNIQUE_SUBCAT', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, 'SEMESTER', 'CUST_NUM_UNIQUE_SUBCAT', 'CUST', False)
ml_dataset = create_aggregations(df, ml_dataset, 'YEAR', 'CUST_NUM_UNIQUE_SUBCAT', 'CUST', False)

In [12]:
# AVERAGE DAYS SINCE LAST CUSTOMER'S TRANSACTION

df = calculate_rolling_avg(df, 'MONTH', 'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUST')
df = calculate_rolling_avg(df, 'QUARTER', 'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUST')
df = calculate_rolling_avg(df, 'SEMESTER', 'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUST')
df = calculate_rolling_avg(df, 'YEAR', 'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUST')

ml_dataset = create_aggregations(df, ml_dataset, 'MONTH', 'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUST', True)
ml_dataset = create_aggregations(df, ml_dataset, 'QUARTER', 'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUST', True)
ml_dataset = create_aggregations(df, ml_dataset, 'SEMESTER', 'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUST', True)
ml_dataset = create_aggregations(df, ml_dataset, 'YEAR', 'CUST_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUST', True)

In [13]:
''' ############ REVIEW LATER ############
# create an empty column for QTY in the new dataframe
ml_dataset['QTY'] = 0

# group the original dataframe by CUSTOMER_ACCOUNT_NR_MASK, SUBCAT_CD_EXT, and MONTH and calculate the sum of QTY
grouped_df = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'MONTH', 'YEAR'])['QTY'].sum().reset_index()

ml_dataset = create_aggregations(df, ml_dataset, 'MONTH', 'CUST_NUM_UNIQUE_SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'QUARTER', 'CUST_NUM_UNIQUE_SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'SEMESTER', 'CUST_NUM_UNIQUE_SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'YEAR', 'CUST_NUM_UNIQUE_SUBCAT', False)
'''

" ############ REVIEW LATER ############\n# create an empty column for QTY in the new dataframe\nml_dataset['QTY'] = 0\n\n# group the original dataframe by CUSTOMER_ACCOUNT_NR_MASK, SUBCAT_CD_EXT, and MONTH and calculate the sum of QTY\ngrouped_df = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'MONTH', 'YEAR'])['QTY'].sum().reset_index()\n\nml_dataset = create_aggregations(df, ml_dataset, 'MONTH', 'CUST_NUM_UNIQUE_SUBCAT', False)\nml_dataset = create_aggregations(df, ml_dataset, 'QUARTER', 'CUST_NUM_UNIQUE_SUBCAT', False)\nml_dataset = create_aggregations(df, ml_dataset, 'SEMESTER', 'CUST_NUM_UNIQUE_SUBCAT', False)\nml_dataset = create_aggregations(df, ml_dataset, 'YEAR', 'CUST_NUM_UNIQUE_SUBCAT', False)\n"

In [14]:
# AVERAGE BASKET SIZE

df['CUST_NUM_SUBCAT_MONTH'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR', 'MONTH'])['SUBCAT_CD_EXT'].transform(count)
df['CUST_NUM_SUBCAT_MONTH'] = df['CUST_NUM_SUBCAT_MONTH']
df['CUST_AVG_BASKET_SIZE_MONTH'] = df['CUST_NUM_SUBCAT_MONTH'] / df['CUST_NUM_TRANSACTIONS_MONTH']

df['CUST_NUM_SUBCAT_QUARTER'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR', 'QUARTER'])['SUBCAT_CD_EXT'].transform(count)
df['CUST_NUM_SUBCAT_QUARTER'] = df['CUST_NUM_SUBCAT_QUARTER']
df['CUST_AVG_BASKET_SIZE_QUARTER'] = df['CUST_NUM_SUBCAT_QUARTER'] / df['CUST_NUM_TRANSACTIONS_QUARTER']

df['CUST_NUM_SUBCAT_SEMESTER'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR', 'SEMESTER'])['SUBCAT_CD_EXT'].transform(count)
df['CUST_NUM_SUBCAT_SEMESTER'] = df['CUST_NUM_SUBCAT_SEMESTER']
df['CUST_AVG_BASKET_SIZE_SEMESTER'] = df['CUST_NUM_SUBCAT_SEMESTER'] / df['CUST_NUM_TRANSACTIONS_SEMESTER']

df['CUST_NUM_SUBCAT_YEAR'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'YEAR'])['SUBCAT_CD_EXT'].transform(count)
df['CUST_NUM_SUBCAT_YEAR'] = df['CUST_NUM_SUBCAT_YEAR']
df['CUST_AVG_BASKET_SIZE_YEAR'] = df['CUST_NUM_SUBCAT_YEAR'] / df['CUST_NUM_TRANSACTIONS_YEAR']


ml_dataset = create_aggregations(df, ml_dataset, 'MONTH', 'CUST_AVG_BASKET_SIZE', 'CUST', True)
ml_dataset = create_aggregations(df, ml_dataset, 'QUARTER', 'CUST_AVG_BASKET_SIZE', 'CUST', True)
ml_dataset = create_aggregations(df, ml_dataset, 'SEMESTER', 'CUST_AVG_BASKET_SIZE', 'CUST', True)
ml_dataset = create_aggregations(df, ml_dataset, 'YEAR', 'CUST_AVG_BASKET_SIZE', 'CUST', True)

Subcategory features

In [15]:
# TOTAL NUMBER OF ORDERS

df['SUBCAT_NUM_TRANSACTIONS_MONTH'] = df.groupby(['SUBCAT_CD_EXT', 'YEAR', 'MONTH'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['SUBCAT_NUM_TRANSACTIONS_MONTH'] = df['SUBCAT_NUM_TRANSACTIONS_MONTH'].astype(int)

df['SUBCAT_NUM_TRANSACTIONS_QUARTER'] = df.groupby(['SUBCAT_CD_EXT', 'YEAR', 'QUARTER'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['SUBCAT_NUM_TRANSACTIONS_QUARTER'] = df['SUBCAT_NUM_TRANSACTIONS_QUARTER'].astype(int)

df['SUBCAT_NUM_TRANSACTIONS_SEMESTER'] = df.groupby(['SUBCAT_CD_EXT', 'YEAR', 'SEMESTER'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['SUBCAT_NUM_TRANSACTIONS_SEMESTER'] = df['SUBCAT_NUM_TRANSACTIONS_SEMESTER'].astype(int)

df['SUBCAT_NUM_TRANSACTIONS_YEAR'] = df.groupby(['SUBCAT_CD_EXT', 'YEAR'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['SUBCAT_NUM_TRANSACTIONS_YEAR'] = df['SUBCAT_NUM_TRANSACTIONS_YEAR'].astype(int)


ml_dataset = create_aggregations(df, ml_dataset, 'MONTH', 'SUBCAT_NUM_TRANSACTIONS', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'QUARTER', 'SUBCAT_NUM_TRANSACTIONS', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'SEMESTER', 'SUBCAT_NUM_TRANSACTIONS', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'YEAR', 'SUBCAT_NUM_TRANSACTIONS', 'SUBCAT', False)

In [16]:
# UNIQUE NUMBER OF CUSTOMERS WHO BOUGHT FROM A SUBCATEGORY

df['SUBCAT_NUM_UNIQUE_CUST_MONTH'] = df.groupby(['SUBCAT_CD_EXT', 'YEAR', 'MONTH'])['CUSTOMER_ACCOUNT_NR_MASK'].transform(count_unique)
df['SUBCAT_NUM_UNIQUE_CUST_MONTH'] = df['SUBCAT_NUM_UNIQUE_CUST_MONTH'].astype(int)

df['SUBCAT_NUM_UNIQUE_CUST_QUARTER'] = df.groupby(['SUBCAT_CD_EXT', 'YEAR', 'QUARTER'])['CUSTOMER_ACCOUNT_NR_MASK'].transform(count_unique)
df['SUBCAT_NUM_UNIQUE_CUST_QUARTER'] = df['SUBCAT_NUM_UNIQUE_CUST_QUARTER'].astype(int)

df['SUBCAT_NUM_UNIQUE_CUST_SEMESTER'] = df.groupby(['SUBCAT_CD_EXT', 'YEAR', 'SEMESTER'])['CUSTOMER_ACCOUNT_NR_MASK'].transform(count_unique)
df['SUBCAT_NUM_UNIQUE_CUST_SEMESTER'] = df['SUBCAT_NUM_UNIQUE_CUST_SEMESTER'].astype(int)

df['SUBCAT_NUM_UNIQUE_CUST_YEAR'] = df.groupby(['SUBCAT_CD_EXT', 'YEAR'])['CUSTOMER_ACCOUNT_NR_MASK'].transform(count_unique)
df['SUBCAT_NUM_UNIQUE_CUST_YEAR'] = df['SUBCAT_NUM_UNIQUE_CUST_YEAR'].astype(int)


ml_dataset = create_aggregations(df, ml_dataset, 'MONTH', 'SUBCAT_NUM_UNIQUE_CUST', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'QUARTER', 'SUBCAT_NUM_UNIQUE_CUST', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'SEMESTER', 'SUBCAT_NUM_UNIQUE_CUST', 'SUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'YEAR', 'SUBCAT_NUM_UNIQUE_CUST', 'SUBCAT', False)

Customer-Subcategory features

In [17]:
# TOTAL NUMBER OF ORDERS FOR A SUBCATEGORY BY A SPECIFIC CUSTOMER

df['CUSTSUBCAT_NUM_TRANSACTIONS_MONTH'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR', 'MONTH'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['CUSTSUBCAT_NUM_TRANSACTIONS_MONTH'] = df['CUSTSUBCAT_NUM_TRANSACTIONS_MONTH'].astype(int)

df['CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR', 'QUARTER'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER'] = df['CUSTSUBCAT_NUM_TRANSACTIONS_QUARTER'].astype(int)

df['CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR', 'SEMESTER'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER'] = df['CUSTSUBCAT_NUM_TRANSACTIONS_SEMESTER'].astype(int)

df['CUSTSUBCAT_NUM_TRANSACTIONS_YEAR'] = df.groupby(['CUSTOMER_ACCOUNT_NR_MASK', 'SUBCAT_CD_EXT', 'YEAR'])['TRANSACTION_ID_MASK'].transform(count_unique_transactions)
df['CUSTSUBCAT_NUM_TRANSACTIONS_YEAR'] = df['CUSTSUBCAT_NUM_TRANSACTIONS_YEAR'].astype(int)


ml_dataset = create_aggregations(df, ml_dataset, 'MONTH', 'CUSTSUBCAT_NUM_TRANSACTIONS', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'QUARTER', 'CUSTSUBCAT_NUM_TRANSACTIONS', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'SEMESTER', 'CUSTSUBCAT_NUM_TRANSACTIONS', 'CUSTSUBCAT', False)
ml_dataset = create_aggregations(df, ml_dataset, 'YEAR', 'CUSTSUBCAT_NUM_TRANSACTIONS', 'CUSTSUBCAT', False)

In [18]:
# AVERAGE DAYS SINCE LAST CUSTOMER'S TRANSACTION

df = calculate_rolling_avg(df, 'MONTH', 'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUSTSUBCAT')
df = calculate_rolling_avg(df, 'QUARTER', 'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUSTSUBCAT')
df = calculate_rolling_avg(df, 'SEMESTER', 'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUSTSUBCAT')
df = calculate_rolling_avg(df, 'YEAR', 'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUSTSUBCAT')

ml_dataset = create_aggregations(df, ml_dataset, 'MONTH', 'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUSTSUBCAT', True)
ml_dataset = create_aggregations(df, ml_dataset, 'QUARTER', 'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUSTSUBCAT', True)
ml_dataset = create_aggregations(df, ml_dataset, 'SEMESTER', 'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUSTSUBCAT', True)
ml_dataset = create_aggregations(df, ml_dataset, 'YEAR', 'CUSTSUBCAT_AVG_DAYS_SINCE_PRIOR_TRANSACTION', 'CUSTSUBCAT', True)

Other features

In [19]:
customer = fill_missing_values(df)  # Fills the missing values of the customer column

In [20]:
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
customer_dict = dict(zip(customer['CUSTOMER_ACCOUNT_NR_MASK'], customer['GENDER']))

# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
ml_dataset['GENDER'] = ml_dataset['CUSTOMER_ACCOUNT_NR_MASK'].map(customer_dict)

In [21]:
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
family_dict = dict(zip(customer['CUSTOMER_ACCOUNT_NR_MASK'], customer['FAMILY_MEMBERS']))

# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
ml_dataset['FAMILY_MEMBERS'] = ml_dataset['CUSTOMER_ACCOUNT_NR_MASK'].map(family_dict)

In [22]:
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
lifestyle_dict = dict(zip(customer['CUSTOMER_ACCOUNT_NR_MASK'], customer['seg_lifestyle_cd']))

# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
ml_dataset['SEG_LIFESTYLE_CD'] = ml_dataset['CUSTOMER_ACCOUNT_NR_MASK'].map(lifestyle_dict)

In [23]:
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
lifestage_dict = dict(zip(customer['CUSTOMER_ACCOUNT_NR_MASK'], customer['seg_lifestage_cd']))

# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
ml_dataset['SEG_LIFESTAGE_CD'] = ml_dataset['CUSTOMER_ACCOUNT_NR_MASK'].map(lifestage_dict)

In [24]:
query = client.query("""
   SELECT SUBCAT_CD_EXT, CAT_CD_EXT, PRICE_RANGE
   FROM tables_raw.dim_product
   """)

products = query.result().to_dataframe() # Wait for the job to complete.

In [25]:
# create a dictionary that maps CUSTOMER_ACCOUNT_NR_MASK values to their corresponding gender values
lifestage_dict = dict(zip(products['SUBCAT_CD_EXT'], products['CAT_CD_EXT']))

# map the gender values to the CUSTOMER_ACCOUNT_NR_MASK column in result_df using the customer_dict mapping
ml_dataset['CAT_CD_EXT'] = ml_dataset['SUBCAT_CD_EXT'].map(lifestage_dict)

Target features

In [26]:
ml_dataset = ml_dataset.sort_values(['CUSTOMER_ACCOUNT_NR_MASK','SUBCAT_CD_EXT','YEAR','MONTH'])
ml_dataset = compute_target(ml_dataset)

# Load dataset into BigQuery

In [28]:
# Shuffle the dataframe
ml_dataset = ml_dataset.sample(frac=1).reset_index(drop=True)

MemoryError: Unable to allocate 1.43 GiB for an array with shape (9, 21278592) and data type int64

In [35]:
#### SAVE DATAFRAME TO BIGQUERY ####
client.load_table_from_dataframe(ml_dataset, 'tables_staging.df_models').result()

LoadJob<project=continente-lced-feup, location=europe-southwest1, id=dca1c420-968d-41cf-ad6d-284d50cfa12e>